In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

22/08/21 08:28:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

22/08/21 08:28:46 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
lines.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [5]:
words.printSchema()

root
 |-- word: string (nullable = true)



In [6]:
wordCounts = words.groupBy("word").count()

In [8]:
query = wordCounts \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()


22/08/21 08:29:12 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/q0/_f4zx8bs08z35px0kj37zldc0000gn/T/temporary-c7de5fc5-a3e7-4333-8aae-2f8bfa2df9dd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|       for|    1|
| streaming|    2|
|        is|    1|
|     spark|    2|
|structured|    1|
|      from|    1|
|      this|    1|
|   another|    1|
|      line|    1|
+----------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|spark|    3|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|     is|    2|
|   this|    2|
|another|    2|
+-------+-----+



22/08/21 08:30:04 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


In [ ]:
query.awaitTermination()

22/08/21 11:08:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 5960511 ms exceeds timeout 120000 ms
22/08/21 11:08:35 WARN SparkContext: Killing executors is not supported by current scheduler.
